In [26]:
import pandas as pd
import lightgbm
import timeit

In [28]:
model_path = './NiR4_OFE_LGBM_model.txt'
data_path = './File_0-csv.txt'
data = pd.read_csv(data_path, header=None)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,4854,4855,4856,4857,4858,4859,4860,4861,4862,4863
0,129,612.0,153000.0,6000.0,0.0,0.0,118470648.0,0.0,628.0,6000.0,...,0.0,132.0,140.0,137.0,135.0,0.0,0.0,0.0,0.0,0.0
1,120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,76,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
booster = lightgbm.Booster(model_file=model_path)

inference_data = lightgbm.Dataset(data_path, free_raw_data=False).construct()
inference_raw_data = inference_data.get_data()

[LightGBM] [Info] Construct bin mappers from text data time 2.14 seconds


In [43]:
predictions_array_lgbmpython = booster.predict(
    data=inference_raw_data,
    num_threads=1,
    predict_disable_shape_check=True,
)
predictions_array_lgbmpython

array([ 1.41436096, -0.27520206, -0.32896408, ...,  0.27021392,
        0.06719871,  2.11317219])

In [37]:
import treelite, treelite_runtime

model = treelite.Model.load(
    model_path,
    model_format="lightgbm"
)
model.export_lib(
    toolchain="msvc",
    libpath=model_path + ".so",
    verbose=True,
    params={'parallel_comp':16}
)

[13:57:51] D:\a\1\s\src\compiler\ast_native.cc:711: Using ASTNativeCompiler
[13:57:51] D:\a\1\s\src\compiler\ast\split.cc:29: Parallel compilation enabled; member trees will be divided into 16 translation units.
[13:57:55] D:\a\1\s\src\c_api\c_api.cc:92: Code generation finished. Writing code to files...
[13:57:55] D:\a\1\s\src\c_api\c_api.cc:97: Writing file tu2.c...
[13:57:55] D:\a\1\s\src\c_api\c_api.cc:97: Writing file main.c...
[13:57:55] D:\a\1\s\src\c_api\c_api.cc:97: Writing file header.h...
[13:57:55] D:\a\1\s\src\c_api\c_api.cc:97: Writing file tu5.c...
[13:57:55] D:\a\1\s\src\c_api\c_api.cc:97: Writing file tu0.c...
[13:57:55] D:\a\1\s\src\c_api\c_api.cc:97: Writing file tu1.c...
[13:57:55] D:\a\1\s\src\c_api\c_api.cc:97: Writing file tu3.c...
[13:57:55] D:\a\1\s\src\c_api\c_api.cc:97: Writing file tu4.c...
[13:57:55] D:\a\1\s\src\c_api\c_api.cc:97: Writing file tu6.c...
[13:57:55] D:\a\1\s\src\c_api\c_api.cc:97: Writing file tu7.c...
[13:57:55] D:\a\1\s\src\c_api\c_api.cc:9

In [38]:
predictor = treelite_runtime.Predictor(
    model_path + '.so',
    verbose=True,
    nthread=1
)
dmat = treelite_runtime.DMatrix(data.to_numpy())

[13:59:32] c:\Users\damajercak\.conda\envs\lightgbmbenchmark\lib\site-packages\treelite_runtime\predictor.py:159: Dynamic shared library c:\Projects\lightgbm-benchmark\NiR4_OFE_LGBM_model.txt.so has been successfully loaded into memory


In [40]:
predictions_array_treelite =predictor.predict(dmat)

In [58]:
from onnxconverter_common.data_types import FloatTensorType
from onnxmltools.convert import convert_lightgbm
import onnxruntime as ort
import numpy as np

with open(model_path, "r") as mf:
    model_str = mf.read()
    model_str = model_str.replace(
        "objective=lambdarank", "objective=regression"
    )
booster_ort = lightgbm.Booster(model_str=model_str)

onnx_input_types = [
    (
        "input",
        FloatTensorType(
            [1, inference_data.num_feature()]
        ),
    )
]
onnx_input_batch_types = [
    (
        "input",
        FloatTensorType(
            [inference_data.num_data(), inference_data.num_feature()]
        ),
    )
]
onnx_ml_model = convert_lightgbm(booster_ort, initial_types=onnx_input_types)
onnx_ml_batch_model = convert_lightgbm(booster_ort, initial_types=onnx_input_batch_types)

sess_options = ort.SessionOptions()
sess_options.intra_op_num_threads = 0
sess_options.inter_op_num_threads = 0

sess_options.execution_mode = (
    ort.ExecutionMode.ORT_SEQUENTIAL
)
sess_options.graph_optimization_level = (
    ort.GraphOptimizationLevel.ORT_ENABLE_ALL
)
sessionml = ort.InferenceSession(
    onnx_ml_model.SerializeToString(), sess_options
)
sessionml_batch = ort.InferenceSession(
    onnx_ml_batch_model.SerializeToString(), sess_options
)
inference_data = lightgbm.Dataset(
    data_path, free_raw_data=False
).construct()
inference_raw_data = inference_data.get_data()
if type(inference_raw_data) == str:
    inference_raw_data = np.loadtxt(
        inference_raw_data, delimiter=","
    ).astype(np.float32)[:, : inference_data.num_feature()]

The maximum opset needed by this model is only 8.
The maximum opset needed by this model is only 8.


[LightGBM] [Info] Construct bin mappers from text data time 2.43 seconds


In [59]:
predictions_array_ort = sessionml_batch.run(
    [sessionml.get_outputs()[0].name],
    {sessionml.get_inputs()[0].name: inference_raw_data},
)[0][:, 0]

In [72]:
time_inferencing_per_query = []
for i in range(len(inference_raw_data)):
    prediction_time = timeit.timeit(
        lambda: sessionml.run(
            [sessionml.get_outputs()[0].name],
            {sessionml.get_inputs()[0].name: inference_raw_data[i : i + 1]},
        ),
        number=1,
    )
    time_inferencing_per_query.append(prediction_time/1)
sum(time_inferencing_per_query)

0.7666397998491448

In [71]:
timeit.timeit(lambda: sessionml_batch.run(
    [sessionml.get_outputs()[0].name],
    {sessionml.get_inputs()[0].name: inference_raw_data},
), number=1)

0.400464499998634

In [52]:
print(predictions_array_lgbmpython[:10])
print(predictions_array_treelite[:10])
print(predictions_array_ort[:10])

[ 1.41436096 -0.27520206 -0.32896408 -0.08315643 -0.26660063 -0.30202675
 -0.22120572 -0.35424621 -0.25634644 -0.06725079]
[ 1.7355299   0.2582493   0.28444618  0.51784474  0.49668223 -0.04218447
  0.12811233  0.20044815 -0.10399695  0.61548153]
[ 1.7355288   0.25824943  0.28444648  0.5178444   0.49668252 -0.04218467
  0.12811226  0.20044814 -0.10399713  0.6154818 ]
